# 3-9 Non-Standard Constraints
* Setting up
* Phase 1
* Phase 2


In [1]:
using Revealables
include("files/answers.jl")

T<:Real}, IntervalArithmetic.Interval{S<:Real}) in module IntervalArithmetic at C:\Users\Victoria Docherty\.julia\v0.6\IntervalArithmetic\src\intervals\arithmetic.jl:26 overwritten in module ImplicitEquations at C:\Users\Victoria Docherty\.julia\v0.6\ImplicitEquations\src\intervals.jl:9.


##Maximizing with Mixed Constraints
Some maximization problems contain mixed constraints, like this:

maximize $3x_1 + 2x_2$ 

subject to 
$\begin{array}{cc}
2x_1 + x_2 \leq 50 & (standard) \\
x_1 + 3x_2 \geq 15 & (greater~than) \\
5x_1 + 6x_2 = 60 & (equality)
\end{array}$

These can come in any combination, but in a typical problem more constraints are standard than nonstandard.

The first thing that needs to be done is to convert all constraints into equality constraints:

$\begin{array}{ccc}
2x_1 + x_2 ≤ 50 &	becomes &	2x_1 + x_2 + x_3 = 50 \\
x_1 + 3x_2 ≥ 15 &	becomes &	x_1 + 3x_2 – x_4 = 15 \\
5x_1 + 6x_2 = 60 &	remains &	5x_1 + 6x_2 = 60
\end{array}$

But this causes some problems in the initial tableau.

##Problems With Mixed Constraints
If written directly into the tableau, the modified constraints and objective function create:

$\begin{array}{c}
\begin{array}{cccccc}
x_1 & x_2 &x_3 &x_4 & f & ans
\end{array} \\
\left[ \begin{array}{cccccc}
2 & 1 & 1 & 0 & 0 & 50 \\
1 & 3 & 0 & \color{orange}{-1} & \color{orange}{0} & \color{orange}{15} \\
\color{red}{5} & \color{red}{6} & \color{red}{0} & \color{red}{0} & \color{red}{0} & \color{red}{60} \\
-3 & -2 & 0 & 0 & 1 & 0
\end{array} \right]
\end{array}$

* <font color="orange">The orange solution is negative and therefore illegal.</font>
* <font color="red">The red row has no solution at all!</font>

##Fixing the Problem
To fix both problem rows, we need to add yet another variable to each one. It looks like this:

$x_1 + 3x_2 ≥ 15$ was $x_1 + 3x_2 – x_4 = 15$;

* now becomes $x_1 + 3x_2 – x_4 + x_5 = 15$
    
$5x_1 + 6x_2 = 60$ did not change;

* now becomes $5x_1 + 6x_2 + x_6 = 0$
    
The variables $x_5$ and $x_6$ are called "artificial" variables because their value must equal zero. It is important to keep track of them in the tableau, so I marked them with asterisks (\*).

When you put the new, improved constraints into the tableau, it will look like this:

$\begin{array}{c}
\begin{array}{cccccccc}
x_1 & x_2 &x_3 &x_4 & x_5^* & x_6^* & f & ans
\end{array} \\
\left[ \begin{array}{cccccccc}
2 & 1 & \color{red}{1} & \color{red}{0} & 0 & 0 & 0 & 50 \\
1 & 3 & \color{red}{0} & \color{red}{-1} & 1 & 0 & 0 & 15 \\
5 & 6 & \color{red}{0} & \color{red}{0} & 0 & 1 & 0 & 60 \\
 & & & & & & &
\end{array} \right]
\end{array}$

If you <font color="red">switch the columns for $x_3$ and $x_4$</font>, you can almost completely fix the problem! Unfortunately, we have a new problem: by definition, $x_5$ and $x_6$ must be 0, and right now they're not. Plus, $x_4$ is still illegal.

The first thing we do is minimize $x_5$ and $x_6$; their minimum value will be 0, which is what we want. We do this by introducing a new, temporary objective function:

minimize $g = x_5 + x_6$.

This is the same as maximizing $g = -x_5 - x_6$, or $\color{blue}{x_5 + x_6 + g = 0}$.

This creates the tableau:

$\begin{array}{c}
\begin{array}{cccccccc}
x_1 & x_2 &x_3 &x_4 & x_5^* & x_6^* & g & ans
\end{array} \\
\left[ \begin{array}{cccccccc}
2 & 1 & 1 & 0 & 0 & 0 & 0 & 50 \\
1 & 3 & 0 & -1 & 1 & 0 & 0 & 15 \\
5 & 6 & 0 & 0 & 0 & 1 & 0 & 60 \\
0 & 0 & 0 & 0 & \color{blue}{1} & \color{blue}{1} & \color{blue}{1} & \color{blue}{0}
\end{array} \right]
\end{array}$

and new problems: no negative indicators.

If you only have one problem row, use (objective row) – (problem row); if you have more than two problem rows, add them all.

This also took care of our $x_4$ problem; it is no longer in the basis.

For the last fix, we need to replace the objective function row. To do that, we will <font color="green">find the sum of both problem rows and subtract that from the objective row</font>:

$\begin{array}{cc}
 & \begin{array}{cccccccc}
x_1 & x_2 &x_3 &x_4 & x_5^* & x_6^* & g & ans
\end{array} \\
from & \left[ \begin{array}{cccccccc}
2 & 1 & 1 & 0 & 0 & 0 & 0 & 50 \\
\color{green}{1} & \color{green}{3} & \color{green}{0} & \color{green}{-1} & \color{green}{1} & \color{green}{0} & \color{green}{0} & \color{green}{15} \\
\color{green}{5} & \color{green}{6} & \color{green}{0} & \color{green}{0} & \color{green}{0} & \color{green}{1} & \color{green}{0} & \color{green}{60} \\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 0
\end{array} \right] \\
 & \\
to & \left[ \begin{array}{cccccccc}
2 & 1 & 1 & 0 & 0 & 0 & 0 & 50 \\
1 & 3 & 0 & -1 & 1 & 0 & 0 & 15 \\
5 & 6 & 0 & 0 & 0 & 1 & 0 & 60 \\
-6 & -9 & 0 & 1 & 0 & 0 & 1 & \color{red}{-75}
\end{array} \right]
\end{array}$

That negative in the "answer" column is appropriate. Because we're maximizing a negative, we should expect a negative value for our answer.

Woohoo! No problems!

That means we can finally start pivoting....

##Summary of Setup Steps
The steps for setting up, in order:
1. Convert inequality constraints to equality constraints. 
2. Add artificial variables to any constraints with a subtracted slack variable, or no slack variable.
3. Write an objective function that minimizes your artificial variables (from step 2) and put it all in a tableau.
4. Replace the last row of the tableau by adding all the problem rows and subtracting that from the last row.

###Practice Problem A
Convert this problem into a proper initial tableau.

Maximize $f = 10x_1 + 20x_2$

subject to $\begin{align}
4x_1 + 5x_2 &= 240 \\
2x_1 + x_2 &≤ 90 \\
x_1 + 3x_2 &≥ 120
\end{align}$

Note which variables are artificial, with a circle or star or by using a different letter to label the column.


In [ ]:
revealable(ans309A)

##Phase 1
With nonstandard constraints, setting up the initial tableau is the hardest part. Once you have that, you can pivot as usual.

After pivoting twice from the initial tableau, we have:
* <font color="red">no negative indicators.</font>
* <font color="blue">artificial variables no longer in the basis, so they = 0.</font>

$\left[ \begin{array}{cccccccc}
0 & 0 & 27 & -21 & \color{blue}{21} & \color{blue}{-15} & 0 & 765 \\
0 & 27 & 0 & -15 & \color{blue}{15^*} & \color{blue}{-3} & 0 & 45 \\
9 & 0 & 0 & 6 & \color{blue}{-6} & \color{blue}{3^*} & 0 & 90 \\
\color{red}{0} & \color{red}{0} & \color{red}{0} & \color{red}{0} & \color{purple}{27} & \color{purple}{27} & \color{red}{27} & \color{red}{0}
\end{array} \right]$

So what was that all about, anyway?
* The simplex method always starts at the origin. 
* In standard problems, the origin is always a corner of the feasible region.
* In problems with nonstandard constraints (like = or ≥), the origin is typically not in the feasible region.
* So, the first step is to *__get to the feasible region__* so we can maximize from there.
This is what the procedure in Phase 1 accomplished.

Now we're ready for Phase 2!

###Practice Problem B
Use your pivoting program to complete Phase 1 on the tableau from Practice Problem A.

In [ ]:
# Code here!

In [ ]:
revealable(ans309B)

##Phase 2
After phase 1, we are in the feasible region, but we still need to maximize the original objective function. In the example it was $3x_1 + 2x_2$.

$\left[ \begin{array}{cccccccc}
0 & 0 & 27 & -21 & \color{blue}{21} & \color{blue}{-15} & 0 & 765 \\
0 & 27 & 0 & -15 & \color{blue}{15^*} & \color{blue}{-3} & 0 & 45 \\
9 & 0 & 0 & 6 & \color{blue}{-6} & \color{blue}{3^*} & 0 & 90 \\
\color{red}{0} & \color{red}{0} & \color{red}{0} & \color{red}{0} & \color{purple}{27} & \color{purple}{27} & \color{red}{27} & \color{red}{0}
\end{array} \right]$

* <font color="blue">Delete the columns containing artificial variables</font> 
* <font color="red">Fill in the original objective function in the bottom row</font>:

$\left[ \begin{array}{cccccccc}
0 & 0 & 27 & -21 & 0 & 765 \\
0 & 27 & 0 & -15 & 0 & 45 \\
9 & 0 & 0 & 6 & 0 & 90 \\
\color{red}{-3} & \color{red}{-2} & \color{red}{0} & \color{red}{0} & \color{red}{1} & \color{red}{0}
\end{array} \right]$


Finally, pivot as usual from this tableau until all negative indicators are gone. The solution to this problem is a maximum value of $33.33$ when $x_1 = 10$, $x_2 = 1.67$, and $x_3 = 28.33$.

###Practice Problem C
Complete Phase 2 on your matrix from Practice Problem B; the original objective function was to maximize $10x_1 + 20x_2$.

In [ ]:
revealable(ans309C)

##Nonstandard Constraints: All the Steps
1. Modify all constraints by adding or subtracting slack variables and adding artificial variables as needed.
2. Set up a matrix to minimize the sum of the artificial variables. Subtract (bottom row) – (sum of all problem rows) to get the phase 1 matrix.
3. Pivot as usual to maximize the phase 1 matrix.
4. Replace the objective row with the actual objective function and cross off the artificial variable columns.
5. Maximize as usual.

###Practice Problem D
Complete this problem from start to finish:

Maximize $f = 15x_1 + 10x_2 + 20x_3$

subject to	$\begin{align}
2x_1 + 4x_2 + x_3 &≤ 20 \\
3x_1 + x_2 + 5x_3 &≥ 10
\end{align}$


Space for writing:


In [ ]:
# Space for coding/calculating:


In [ ]:
revealable(ans309D)

##Note on Minimization
The actual procedures for minimization with nonstandard constraints (≤ or =) are complex.

More commonly, the initial tableau is set up and various pivot candidates are tried, with the goal of reaching the feasible region. After the feasible region is attained (in other words, all illegal solutions leave the basis and no new ones enter), the tableau can be optimized from there.